In [1]:
import numpy as np
import pandas as pd
import os
import re
from PIL import Image

from pathlib import Path
import json
import shutil

## Converting VoTT output

After labeling with VoTT, we obtain a .csv file, which we convert into many .txt files, one file for every frame. These will be saved in a folder called labels_with_ids (if it does not exist you have to create it.

### Paths to everything

#### Paths Richards things

In [3]:
#path to image and label folder
data_root = Path("/usr/users/vogg/Labelling/Lemurs/LemurBoxTracking/LemurBoxApr23/")
#data_root = Path("/usr/users/agecker/datasets/MacaqueImagePairs/")
labels = pd.read_csv(data_root / "LemurBoxApr23-export.csv")
labels.head()

,image,xmin,ymin,xmax,ymax,label
0,B-2023-05-08-GX010030_cam4_00001.jpg,777.345201,549.056766,1216.764706,871.655677,box
1,B-2023-05-08-GX010030_cam4_00001.jpg,195.487616,455.601778,497.515480,702.526376,box
2,B-2023-05-08-GX010030_cam4_00001.jpg,168.769350,264.821388,289.334365,527.622420,lemur
3,B-2023-05-08-GX010030_cam4_00002.jpg,422.809598,419.521789,897.786378,774.407683,lemur
4,B-2023-05-08-GX010030_cam4_00002.jpg,180.835913,453.039564,487.770898,704.763475,box


### Get all images in a list

In [5]:
img_list = os.listdir(data_root / "images")
img_list = [img for img in img_list if not img.startswith(".")]
print(len(img_list))

1919


### Extracting multiple json VoTT output for single-class tracking

- used for testing Zurnas Explorationroom labeling with single class FairMOT 
- all classes labeled in VoTT but only monkey class BB's are extracted

In [ ]:
# cell to extract monkey class
i_monkey = 0
for label in labels_list:
    # print(label)
    with open(label) as f:
        content = json.load(f)
        
    origin_path = Path(content["asset"]["path"])
    image_path_end = Path(origin_path.parent.name, origin_path.name)
    image_path = data_root / label.parent.parent.name  / image_path_end
    # print(image_path)
    # img = np.asarray(Image.open(image_path))
    # img_h, img_w, _ = img.shape
    img_h, img_w = content["asset"]["size"].values()

    all_boxes = content["regions"]
    # print(all_boxes[0])
    monkey_boxes = [box for box in all_boxes if "monkey" in box["tags"]]
    # if monkey_boxes:
    #     shutil.copy(image_path, (data_root / 'images' /image_path.name))
    # print(monkey_boxes)

    label_fpath = (data_root / "labels_with_ids" / image_path.name).with_suffix('.txt')
    # print(label_fpath)

    for monkey_box in monkey_boxes:
        id = monkey_box["id"]
        h, w, left, top = monkey_box["boundingBox"].values()
        # print(h,w,left,top)

        x_center = left + w/2
        y_center = top + h/2

        #  img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.

        label_str = '0 {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                i_monkey, x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        i_monkey += 1
        with label_fpath.open(mode='a') as f:
            f.write(label_str)
    # plotBBox(str(image_path.stem),str(image_path.parent)+'/',str(label_fpath.parent)+'/', ending='.png')
        


### Extracting multiple json VoTT output for multi-class FairMOT

- used for Zurnas Explorationroom labeling

In [68]:
# cell for all classes
i_obj = [0,0,0,0,0]

for label in labels_list:
    # print(label)
    with open(label) as f:
        content = json.load(f)
        
    origin_path = Path(content["asset"]["path"])
    image_path_end = Path(origin_path.parent.name, origin_path.name)
    image_path = data_root / label.parent.parent.name  / image_path_end
    # print(image_path)
    # img = np.asarray(Image.open(image_path))
    # img_h, img_w, _ = img.shape
    img_h, img_w = content["asset"]["size"].values()

    all_boxes = content["regions"]

    # monkey_boxes = [box for box in all_boxes if "monkey" in box["tags"]]

    label_fpath = (data_root / "labels_with_ids" / image_path.name).with_suffix('.txt')

    for box in all_boxes:
        if "monkey" in box["tags"]:
            obj_class = 0
        elif "patch" in box["tags"]:
            obj_class = 1
        elif "kong" in box["tags"]:
            obj_class = 2
        elif "branch" in box["tags"]:
            obj_class = 3
        elif "XBI" in box["tags"]:
            obj_class = 4
        h, w, left, top = box["boundingBox"].values()
        # print(h,w,left,top)

        x_center = left + w/2
        y_center = top + h/2

        #  img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.

        label_str = '{:d} {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                obj_class, i_obj[obj_class], x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        i_obj[obj_class] += 1
        with label_fpath.open(mode='a') as f:
            f.write(label_str)

### Extracting one csv VoTT output for multi-class tracking

- used for lemurs and boxes

In [12]:
i_obj = [0,0]
img_paths = []
for _, row in labels.iterrows():
    # print(label)
    image, xmin, ymin, xmax, ymax, label = row

    image_path = data_root / "images" / image
    img_paths.append(str(image_path))
    # print(image_path)
    img = np.asarray(Image.open(image_path))
    img_h, img_w, _ = img.shape

    # for lemur and box ---
    # img_h, img_w = [2160, 3840]


    # label_fpath = (data_out / 'labels' / image_path.name).with_suffix('.txt')
    # for exp room
    if not os.path.exists(data_root / 'labels_with_ids' ):
        os.makedirs(data_root / 'labels_with_ids' )

    label_fpath = (data_root / 'labels_with_ids' / image_path.name).with_suffix('.txt')
    
    #obj_class = 0

    #for lemur and box ---
    if "lemur" in label:
        obj_class = 0
    elif "box" in label:
        obj_class = 1

    w = (xmax - xmin)
    h = (ymax - ymin)

    x_center = xmin + w / 2
    y_center = ymin + h / 2

    #  img_id = re.sub("[.]jpg","", img_id)
    #Label-String schreiben.

    label_str = '{:d} {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
            obj_class, i_obj[obj_class], x_center / img_w, y_center / img_h, w / img_w, h / img_h)

    i_obj[obj_class] += 1

    # not for lemur box
    with label_fpath.open(mode='a') as f:
        f.write(label_str)

# for lemur box
# img_paths = list(set(img_paths))
# with (data_out / 'lemur_box.train').open('w') as f:
#     for ip in img_paths:
#         f.write(f'{ip}\n')

### Extracting one csv VoTT output for multi-class FairMOT + classification

In [11]:
dict_indivs = {'Cha': 0, 'Flo': 1, 'Gen': 2, 'Geo': 3, 'Her': 4, 'Rab': 5, 'Red': 6, 'Uns': 7}

In [14]:
i_obj = [0,0]
img_paths = []
for _, row in labels.iterrows():
    # print(label)
    image, xmin, ymin, xmax, ymax, label = row
    #indiv = label.split("-")[1][:3]
    indiv = label[:3]

    image_path = data_root / 'eval' / image
    img_paths.append(str(image_path))
    # print(image_path)
    img = np.asarray(Image.open(image_path))
    img_h, img_w, _ = img.shape

    # for lemur and box ---
    # img_h, img_w = [2160, 3840]


    # label_fpath = (data_out / 'labels' / image_path.name).with_suffix('.txt')
    # for exp room
    label_fpath = (data_root / 'labels_with_ids' / image_path.name).with_suffix('.txt')
    
    obj_class = 0

    # for lemur and box ---
    if "Lemur" in label:
        obj_class = 0
    elif "Box" in label:
        obj_class = 1

    w = (xmax - xmin)
    h = (ymax - ymin)

    x_center = xmin + w / 2
    y_center = ymin + h / 2

    #  img_id = re.sub("[.]jpg","", img_id)
    #Label-String schreiben.

    label_str = '{:d} {:d} {:.6f} {:.6f} {:.6f} {:.6f} {:d}\n'.format(
            obj_class, i_obj[obj_class], x_center / img_w, y_center / img_h, 
        w / img_w, h / img_h, dict_indivs[indiv])

    i_obj[obj_class] += 1

    # not for lemur box
    with label_fpath.open(mode='a') as f:
        f.write(label_str)
    



# Original VoTT output transform for FairMOT (from Richard)

- I at least think that that is in this cell ^^

In [332]:
i_monkey = 0

# for img_id in img_list:
for img_path in img_list:
    idx = np.where(labels.image == img_id)
    image_labels = labels.iloc[idx]
    
    label_fpath = data_root + "labels_with_ids/" + re.sub(".JPG", ".txt", img_id)

    img = np.asarray(Image.open(data_root + "images/" + img_id))
    img_h, img_w, _ = img.shape

    for index, row in image_labels.iterrows():

        #if row.label == "baboon":
        x_center = row.xmin + (row.xmax - row.xmin)/2
        y_center = row.ymin + (row.ymax - row.ymin)/2
        w = row.xmax - row.xmin
        h = row.ymax - row.ymin

        #i_monkey = row.label

        img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.

        label_str = '0 {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                i_monkey, x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        i_monkey += 1
        with open(label_fpath, 'a') as f:
            f.write(label_str)


            

# Converting CVAT output

Data comes in separate files.

In [353]:
video_id = 'VID_20210227_133440_0'

data_root = "/usr/users/agecker/datasets/macaque_videos/"

path_o = data_root + video_id + "/obj_train_data/"

frame_list = os.listdir(path_o)
frame_list.sort()

In [354]:
regex = re.compile('.*txt')
frame_list = [i for i in frame_list if regex.match(i)]
print(len(frame_list))

361


In [343]:

!mkdir /usr/users/agecker/datasets/macaque_videos/VID_20210228_153846_0/labels_with_ids/

mkdir: cannot create directory ‘/usr/users/agecker/datasets/macaque_videos/VID_20210228_153846_0/labels_with_ids/’: File exists


In [356]:
!mkdir /usr/users/agecker/datasets/macaque_videos/VID_20210227_133440_0/images/

In [357]:
for frame_id in frame_list:
    text = pd.read_csv(path_o + frame_id, names = [1,2,3,4,5], sep = " ")
    text.insert(0, "z", np.zeros(len(text)))
    text.z = text.z.astype(int)
    text.to_csv(data_root + video_id + "/labels_with_ids/" + frame_id,
                 sep=' ', index=False, header = False)

## Make files for training list

In [2]:
data_root = Path("/usr/users/vogg/Labelling/Lemurs/Individual_imgs/cleaned_data1/")
label_list = [item for item in os.listdir(data_root / "labels_with_ids") if not item.startswith(".")]
print(len(label_list))
print(label_list[:5])

467
['a_e_3_220920_c2_6966.txt', 'a_e_1_220918_c2_13859.txt', 'a_e_3_220920_c2_15752.txt', 'a_e_3_220920_c2_21930.txt', 'a_e_1_220918_c1_26158.txt']


In [3]:
#/usr/users/vogg/monkey-tracking-in-the-wild/src/data/

for img_id in label_list:
    if not img_id.startswith("."):
        label_fpath = data_root / "lemur_ids_cleaned1.train"

        label_str = data_root.name + "/images/" + img_id.replace("txt", "jpg") + "\n"

        with open(label_fpath, 'a') as f:
            f.write(label_str)

## Videos to frames

In [358]:
import cv2
import sys
#sys.path

In [359]:
path = "/usr/users/agecker/datasets/macaque_videos/"
cap = cv2.VideoCapture(path + "Videos/" + video_id + ".mp4")

#Total number of frames
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame count:', frame_count)

count = 0

Frame count: 361


In [360]:
#%%

while count < frame_count:
    cap.set(cv2.CAP_PROP_POS_FRAMES, count)
    success, image = cap.read()
    cv2.imwrite(path + video_id + "/images/frame_%s.jpg" % str(count).zfill(6), image)     # save frame as JPEG file
    count += 1
    if count % 50 == 0:
        print(count)

50
100
150
200
250
300
350
